# Main Thesis Topic: “Zero-shot classification of ECG signals using CLIP-like model”.

**For example: Train on PBT-XL:**

- Text Encoder: ClinicalBERT (trained on diagnoses of ECG signal to obtain corresponding embeddings)
- Image Encoder: 1D-CNN (used to encode ECG signal to obtain signal embeddings)

- Experiment A): Baseline: We can take only the name of the class. For example, take “Myocardial Infarction” as a text. We should exclude some classes from training and after training is completed, the CLIP-like model can be tested on these excluded classes.
    - Next, we get embeddings of text from ClinicalBERT and train the ECG encoder with contrastive loss.

- Experiment B): Same as Experiment A but instead of testing on the same dataset/classes, we would test on other datasets containing different classes.

**Evaluation metrics:**
- Main: AUC-ROC, average_precison_score,
- Optional: Specificity, Sensitivity, F1-score

**Outcome:**
- It’s possible to train CLIP-like models with freezed (or unchanged/not fine tuned for downstream tasks) text encoder
- Training ECG encoders that are viable for representing different domains (within ECG modality) and previously unseen classes.
- Training a CLIP-like model on ECGs has little novelty.

First, we preprocess the ECG data from the PhysioNet 2021 challenge dataset. This data will be loaded using the ```PhysioNetDataset``` class.

In [1]:
pip install transformers

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import random
from tqdm import tqdm
from scipy.signal import resample
import torch
from transformers import AutoTokenizer, AutoModel
import ast
import scipy.io as sio
from torch.utils.data import random_split
from torch.nn.functional import cosine_similarity

In [3]:
import torch
import torch.nn as nn

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
PyFiles_PATH = '/content/drive/MyDrive/ECG Project (Shared Folder)/PyFiles'
PyFiles_PATH

'/content/drive/MyDrive/ECG Project (Shared Folder)/PyFiles'

In [6]:
sys.path.append(PyFiles_PATH)

In [7]:
from helper_functions import *
from dataset import PhysioNetDataset

In [8]:
# Path to training folder within PhysioNet dataset
PhysioNet_PATH = '/content/drive/MyDrive/ECG Project (Shared Folder)/Datasets/physionet.org/files/challenge-2021/1.0.3/training'
PhysioNet_PATH

'/content/drive/MyDrive/ECG Project (Shared Folder)/Datasets/physionet.org/files/challenge-2021/1.0.3/training'

Using the ```PhysioNet_PATH```, we can create separate datasets for training, testing & validation.

# Stage 1: Data Preprocessing

- train_set (train & validation data)
- test_set (test data)

Google CPU is quite slow when processing the for-loops below so if you have a decent CPU, you can try connecting to local run time via Jupyter Lab for this portion of the pipeline only.

```
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0
```

In [9]:
train_set = PhysioNetDataset(PhysioNet_PATH, train=True)
test_set = PhysioNetDataset(PhysioNet_PATH, train=False)

len(train_set), len(test_set)

(66167, 22352)

The ```train_set``` can be split into ```current_train``` (85%) and ```current_val``` (15%).

In [10]:
# Set the seed for the random number generator
torch.manual_seed(0)

# Get the length of the train_set
length = len(train_set)

# Calculate the lengths of the splits
train_length = int(0.85 * length)
val_length = length - train_length

# Split the dataset
current_train, current_val = random_split(train_set, [train_length, val_length])

The next step is to extract the header data for ```current_train```, ```current_val```, and ```test_set``` and save the data to a csv file.

## current_train

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(current_train)), desc="Processing records"):
    record, _ = current_train[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('train_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

## current_val

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(current_val)), desc="Processing records"):
    record, _ = current_val[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('val_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

## test_set

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(test_set)), desc="Processing records"):
    record, _ = test_set[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('test_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

Now that the header data has been extracted and saved to csv files, we can map the corresponding SNOWMED-CT code to the csv files too.

First, let's load the SNOWMED-CT mappings:

In [11]:
smowmed_mappings_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/SNOWMED-CT Codes/combined_mappings.csv'

# Load the SNOMED-CT mappings
smowmed_mappings = pd.read_csv(smowmed_mappings_path)
smowmed_mappings.head(2)

,Dx,SNOMEDCTCode,Abbreviation,CPSC,CPSC_Extra,StPetersburg,PTB,PTB_XL,Georgia,Chapman_Shaoxing,Ningbo,Total,Notes
0,atrial fibrillation,164889003,AF,1221,153,2,15,1514,570,1780,0,5255,NaN
1,atrial flutter,164890007,AFL,0,54,0,1,73,186,445,7615,8374,NaN


In [12]:
# Select the 'Dx' and 'SNOMEDCTCode' columns
codes = smowmed_mappings[['Dx', 'SNOMEDCTCode']]

# Set 'SNOWMEDCTCode' as the index
codes.set_index('SNOMEDCTCode', inplace=True)

# Convert the DataFrame into a dictionary
codes_dict = codes['Dx'].to_dict()

In [ ]:
len(codes_dict)

133

Now, let's load the csv files and map the corresponding codes from ```codes_dict``` to the csv files:

In [13]:
train_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/train_set_records.csv'
val_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/val_set_records.csv'
test_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/test_set_records.csv'

In [14]:
train_set_df = load_and_process(train_set_path)
val_set_df = load_and_process(val_set_path)
test_set_df = load_and_process(test_set_path)

Now, using the ```map_codes_to_dx()``` function, let's map the SNOWMED-CT codes for each ECG signal ```dx```. The new column containing the diagnosis name will be ```dx_modality```

In [ ]:
def map_codes_to_dx(codes):
    return [codes_dict.get(int(code), code) for code in codes]

In [ ]:
train_set_df['dx_modality'] = train_set_df['dx'].apply(map_codes_to_dx)

In [ ]:
val_set_df['dx_modality'] = val_set_df['dx'].apply(map_codes_to_dx)

In [ ]:
test_set_df['dx_modality'] = test_set_df['dx'].apply(map_codes_to_dx)

In [ ]:
# Example:
train_set_df['dx_modality'][0]

['sinus tachycardia',
 't wave inversion',
 't wave abnormal',
 'prolonged qt interval']

Now, let's save these to new csv files so that they contain the new `dx_modality` column:

- `processed_train_set_records.csv`
- `processed_val_set_records.csv`
- `processed_test_set_records.csv`

In [ ]:
train_set_df.to_csv('processed_train_set_records.csv', index=False)

In [ ]:
val_set_df.to_csv('processed_val_set_records.csv', index=False)

In [ ]:
test_set_df.to_csv('processed_test_set_records.csv', index=False)

# Stage 2: ECG Classification Model Pipeline

Now that our data is preprocessed, we can begin working on the Model Pipeline itself. The ECG Classification Model Pipeline will consist of three components:

1. `TextEncoder()` class

2. `ECGEncoder()` class

3. `InstanceSelector()` class

4. `CLIPModel()` class

An overview and outline of each of these components can be found below in their respective subsections.

First, let's load the csv files that contain information about our ECG header data.

`NOTE: The number of records in each csv files should match the number of records in current_train, current_val, and test_set, respectively`

In [15]:
processed_train_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_train_set_records.csv'
processed_val_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_val_set_records.csv'
processed_test_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_test_set_records.csv'

In [16]:
processed_train_df = pd.read_csv(processed_train_set_path)
processed_val_df = pd.read_csv(processed_val_set_path)
processed_test_df = pd.read_csv(processed_test_set_path)

In [17]:
processed_train_df.iloc[0]['dx_modality'], len(processed_train_df.iloc[0]['dx_modality'])

("['sinus tachycardia', 't wave inversion', 't wave abnormal', 'prolonged qt interval']",
 85)

In [18]:
print("There are {} records in the current_train and {} records in the processed_train_df.".format(len(current_train), len(processed_train_df)))

There are 56241 records in the current_train and 56015 records in the processed_train_df.


In [19]:
print("There are {} records in the val_set and {} records in the processed_val_df.".format(len(current_val), len(processed_val_df)))

There are 9926 records in the val_set and 9885 records in the processed_val_df.


In [20]:
print("There are {} records in the test_set and {} records in the processed_test_df.".format(len(test_set), len(processed_test_df)))

There are 22352 records in the test_set and 22352 records in the processed_test_df.


In [21]:
processed_train_df['dx_modality'] = processed_train_df['dx_modality'].apply(ast.literal_eval)
processed_val_df['dx_modality'] = processed_val_df['dx_modality'].apply(ast.literal_eval)
processed_test_df['dx_modality'] = processed_test_df['dx_modality'].apply(ast.literal_eval)

In [22]:
processed_train_df.iloc[0]['dx_modality'], len(processed_train_df.iloc[0]['dx_modality'])

(['sinus tachycardia',
  't wave inversion',
  't wave abnormal',
  'prolonged qt interval'],
 4)

## TextEncoder()

Create a class, ```TextEncoder()``` that is used to convert the description of the (dx_modality) diagnosis class into an embeddings using the pretrained, base ClinicalBERT model.

- Input should be a concatenated using comma or blank space string of diagnoses/dx_modality per ECG signal.
- Use processed CSV files (dx_modality only)
- Frozen weights (since it's already pretrained)

In [23]:
class TextEncoder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        self.model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

    def encode(self, text_list):
        # Check if text_list is a string representation of a list
        if isinstance(text_list, str):
            text_list = ast.literal_eval(text_list)
        # Convert list of strings to a single string
        text = ', '.join(text_list)
        # Tokenize text
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        # Get embeddings from ClinicalBERT model
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state
        # Average the embeddings to get single vector per each input
        embeddings = torch.mean(embeddings, dim=1)
        return embeddings

In [24]:
text_encoder = TextEncoder()
embeddings = text_encoder.encode(processed_train_df['dx_modality'][0])
print(embeddings.size())

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])


In [25]:
embeddings.size()

torch.Size([1, 768])

## ECGEncoder()

- Input is ECG signal, output will be embeddings of ECG signal
- This is going to be model in model.py
- Model weights are updated iteratively
- optimizer = torch.optim.Adam(clip_model.ECGEncoder.parameters())

In [26]:
#import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [27]:
class OneDimCNN(nn.Module):
    def __init__(self, num_classes):
        super(OneDimCNN, self).__init__()

        # Layer 1
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 2
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 3
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 4
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 5
        self.conv5 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm1d(256)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 6
        self.conv6 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm1d(512)
        self.relu6 = nn.ReLU()
        self.pool6 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 7
        self.conv7 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm1d(1024)
        self.relu7 = nn.ReLU()
        self.pool7 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 8
        self.conv8 = nn.Conv1d(in_channels=1024, out_channels=2048, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm1d(2048)
        self.relu8 = nn.ReLU()
        self.pool8 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 9
        self.conv9 = nn.Conv1d(in_channels=2048, out_channels=4096, kernel_size=3, stride=1, padding=1)
        self.bn9 = nn.BatchNorm1d(4096)
        self.relu9 = nn.ReLU()
        self.pool9 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Fully Connected Layer 1
        self.fc1 = nn.Linear(4096, 128)  # Adjusted to match output channels of last conv layer
        self.relu10 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        # Fully Connected Layer 2
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # Layer 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # Layer 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Layer 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool4(x)

        # Layer 5
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.pool5(x)

        # Layer 6
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.pool6(x)

        # Layer 7
        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu7(x)
        x = self.pool7(x)

        # Layer 8
        x = self.conv8(x)
        x = self.bn8(x)
        x = self.relu8(x)
        x = self.pool8(x)

        # Layer 9
        x = self.conv9(x)
        x = self.bn9(x)
        x = self.relu9(x)
        x = x.view(x.size(0), -1)

        # Dynamically adjust the input size of the first fully connected layer
        if self.fc1.in_features != x.size(1):
            self.fc1 = nn.Linear(x.size(1), 128)

        # Fully Connected Layer 1
        x = self.fc1(x)
        x = self.relu10(x)  # Renamed from relu5 to relu10
        x = self.dropout1(x)

        # Fully Connected Layer 2
        x = self.fc2(x)

        return x

In [28]:
class ECGEncoder(OneDimCNN):
    def __init__(self, num_classes, embedding_size=768):
        super(ECGEncoder, self).__init__(num_classes)
        self.fc = nn.Linear(133, embedding_size)  # Add a fully connected layer

    def encode(self, signal):
        encoded_signal = self.forward(signal)
        encoded_signal = self.fc(encoded_signal)  # Transform the embedding to the common size
        return encoded_signal

In [29]:
type(train_set[0][1]['val'])

numpy.ndarray

In [30]:
train_set[0][1]['val']

array([[ -49.41174209,  -49.41174209,  -49.41174209, ...,    3.26986681,
           6.65822132,    4.35320015],
       [  39.23760431,   39.23760431,   39.23760431, ...,  -45.89316397,
         -44.94818514,  -42.39476713],
       [  95.17413779,   95.17413779,   95.17413779, ..., -112.12670284,
        -112.96003617, -116.70843301]])

In [31]:
# Define the number of classes
num_classes = len(codes_dict)

# Create an instance of the model
ecg_encoder = ECGEncoder(num_classes)

# Convert the numpy array to a PyTorch tensor
input_data = torch.from_numpy(train_set[40000][1]['val']).float()

# Add an extra dimension for the batch size
input_data = input_data.unsqueeze(0)
# Convert the model's weights to Float
ecg_encoder = ecg_encoder.float()

# Pass the data through the model
output = ecg_encoder(input_data)

print(output)

tensor([[ 0.3753,  0.2802,  0.1888,  0.6706,  0.4320, -0.2118, -0.1622,  0.1226,
          0.0934,  0.5008, -0.6245, -0.4724,  0.1736,  0.0944, -0.0337,  0.0422,
         -0.1054,  0.0316,  0.3628,  0.2858,  0.1973, -0.0416,  0.1828, -0.2890,
         -0.1381,  0.2132, -0.1095, -0.0270,  0.3953, -0.1959,  0.5568, -0.4805,
         -0.1809, -0.0965, -0.0584,  0.0153, -0.0395, -0.1492,  0.1570, -0.0267,
         -0.3558,  0.1452, -0.2070, -0.1123, -0.0449,  0.0537,  0.3095, -0.0765,
         -0.1267, -0.0273,  0.3605, -0.4415,  0.0839, -0.1570, -0.1314,  0.5673,
         -0.0046,  0.3550, -0.0311, -0.2212,  0.4565,  0.5133,  0.4881,  0.4462,
         -0.1212, -0.1972,  0.3000,  0.5612, -0.1037, -0.4633,  0.1088, -0.1151,
         -0.0785,  0.1945, -0.1743, -0.2203,  0.2623,  0.2241,  0.3988,  0.0170,
          0.0203,  0.0223,  0.3461,  0.2338, -0.9092, -0.0209, -0.3662, -0.1600,
          0.1072,  0.3462, -0.3875, -0.5950, -0.2281, -0.0274, -0.1038, -0.0837,
         -0.2258,  0.0255, -

In [32]:
type(train_set[40000][1]['val'])

numpy.ndarray

In [33]:
batch_size = input_data.shape[0]
num_channels = input_data.shape[1]
tensor_size = input_data[0][2]

print("Batch size:", batch_size)
print("Number of channels:", num_channels)
print(f"Tensor size: {len(tensor_size)}")

Batch size: 1
Number of channels: 3
Tensor size: 5000


In [34]:
output.size()

torch.Size([1, 133])

In [35]:
# Convert the model's weights to Float
ecg_encoder = ecg_encoder.float()

# Set the model in evaluation mode
ecg_encoder.eval()

# Pass the data through the model
output = ecg_encoder(input_data)

print(output)

tensor([[-0.0356,  0.0336, -0.0780,  0.0427,  0.0883,  0.0092, -0.0281,  0.0282,
         -0.0672,  0.0913, -0.0082, -0.0059,  0.0302, -0.0381, -0.0730,  0.0838,
          0.0253,  0.0111,  0.0142,  0.0821,  0.0473, -0.0013, -0.0037, -0.0357,
         -0.0169,  0.0305, -0.0125,  0.0587, -0.0546, -0.0212,  0.0611, -0.0433,
          0.0357, -0.0244, -0.0118, -0.0017, -0.0271, -0.0754,  0.0014,  0.0172,
         -0.0545, -0.0814,  0.0389, -0.0032, -0.0700, -0.0676,  0.0580, -0.0265,
          0.0620, -0.0814, -0.0189, -0.0766, -0.0525,  0.0444, -0.0227,  0.0722,
         -0.0070, -0.0420,  0.0731,  0.0672, -0.0137,  0.0548,  0.0435,  0.0758,
          0.0130,  0.0047,  0.0583,  0.0250,  0.0653, -0.0121, -0.0786, -0.0802,
         -0.0342,  0.0799,  0.0089, -0.0873,  0.0769, -0.0531,  0.0280, -0.0572,
          0.0288, -0.0627,  0.0258,  0.0587, -0.0521,  0.0533,  0.0753, -0.0053,
          0.0193,  0.0799, -0.0849, -0.0596, -0.0320, -0.0823, -0.0026, -0.0285,
         -0.0716,  0.0441,  

## InstanceSelecter()

`InstanceSelector` helps to generate negative instances of embeddings such that pairs of ECG and text embeddings are not from the same record file.

- The `get_negative_instances` method iterates over the dataset and for each record and generates a specified number of negative instances by pairing the ECG embedding of the record with the text embeddings of other records that are not equal to it.

- `num_neg_instances` parameter specifies the number of negative instances to generate for each record. The purpose of this is to drastically reduce computation time. If this was not implemented, there would be too many negative instances:

```len(current_set) - len(current_set_df - 1) total records of negative instances```

- For example, if `num_neg_instances` is set to 5, the method will generate 5 negative instances for each record in the dataset. This is done by randomly selecting 5 records that are not equal to the current record and pairing their text embeddings with the ECG embedding of the current record.

In [36]:
class InstanceSelector:
    def __init__(self, train_set, processed_train_df, text_encoder, ecg_encoder):
        self.train_set = train_set
        self.processed_train_df = processed_train_df
        self.text_encoder = text_encoder
        self.ecg_encoder = ecg_encoder

    def get_negative_instances(self, current_set, current_set_df, num_neg_instances=5):
        # Initialize an empty list to store the negative instances
        negative_instances = []

        # Loop over each record in the current_set
        for i in tqdm(range(len(current_set)), desc="Generating negative instances"):
            # Convert the 'val' field of the current record to a PyTorch tensor and store it in the 'signal' variable
            signal = torch.from_numpy(current_set[i][1]['val']).float()

            # Add an extra dimension to 'signal' to represent the channels
            signal = signal.unsqueeze(0)

            # Encode the 'signal' using the ECG encoder and store the result in the 'ecg_embedding' variable
            ecg_embedding = self.ecg_encoder.encode(signal)

            # Randomly select 'num_neg_instances' indices from current_set_df
            for j in random.sample(range(len(current_set_df)), num_neg_instances):
                # Check if the current index is not the same as the outer loop index
                if i != j:
                    # Encode the 'dx_modality' field of the selected record using the text encoder and store the result in the 'dx_modality_embedding' variable
                    dx_modality_embedding = self.text_encoder.encode(current_set_df['dx_modality'][j])

                    # Check if the ECG embedding is not equal to the DX modality embedding
                    if not torch.all(torch.eq(ecg_embedding.float(), dx_modality_embedding)):
                        # If they are not equal, append the pair (ECG embedding, DX modality embedding) to the 'negative_instances' list
                        negative_instances.append((ecg_embedding, dx_modality_embedding))

        # Return the list of negative instances
        return negative_instances

In [ ]:
text_encoder = TextEncoder()
ecg_encoder = ECGEncoder(num_classes=133)  # Assuming you have this class defined

In [ ]:
len(processed_train_df), len(current_train)

(56015, 56241)

### current_train instances

In [ ]:
instance_selector_train = InstanceSelector(current_train, train_set_df, text_encoder, ecg_encoder)
negative_instances_train = instance_selector_train.get_negative_instances(current_train, processed_train_df)

### current_val instances

In [ ]:
instance_selector_val = InstanceSelector(current_train, train_set_df, text_encoder, ecg_encoder)
negative_instances_val = instance_selector_train.get_negative_instances(current_val, processed_val_df)

## CLIPModel

The final component of the Model Pipeline is to create a `ClIPModel` class which takes `TextEncoder`, `ECGEncoder`, and `InstanceSelector` to train the final model with contrastive loss.  


In [45]:
class CLIPModel(nn.Module):
    def __init__(self, train_set, processed_train_df, num_classes, embedding_size=768):
        super(CLIPModel, self).__init__()
        self.text_encoder = TextEncoder()
        self.ecg_encoder = ECGEncoder(num_classes, embedding_size)
        self.instance_selector = InstanceSelector(train_set, processed_train_df, self.text_encoder, self.ecg_encoder)

    def forward(self, ecgs, diagnoses):
        # Generate embeddings for the ECG signals and diagnoses
        ecg_embeddings = self.ecg_encoder.encode(ecgs)  # I_f
        diagnoses = [diagnosis.to(ecg_embeddings.device) for diagnosis in diagnoses]  # Move diagnoses to the same device as ecg_embeddings
        diagnosis_embeddings = self.text_encoder.encode(diagnoses)  # T_f

        # Joint multimodal embedding
        ecg_embeddings = F.normalize(ecg_embeddings, dim=1)  # I_e
        diagnosis_embeddings = F.normalize(diagnosis_embeddings, dim=1)  # T_e

        # Scaled pairwise cosine similarities
        logits = torch.mm(ecg_embeddings, diagnosis_embeddings.t())  # logits

        # Symmetric loss function
        labels = torch.arange(ecg_embeddings.size(0)).to(ecg_embeddings.device)
        loss_ecg = F.cross_entropy(logits, labels)  # loss_i
        loss_diagnosis = F.cross_entropy(logits.t(), labels)  # loss_t
        loss = (loss_ecg + loss_diagnosis) / 2

        return loss

Example of how the CLIP-like model works:

In [38]:
sample_train_set_250 = [current_train[i] for i in range(250)]
sample_processed_train_df_250 = processed_train_df.iloc[:250]

In [46]:
# Initialize the CLIPModel
clip_model = CLIPModel(sample_train_set_250, sample_processed_train_df_250, num_classes=133, embedding_size=768)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = clip_model.to(device)

# Initialize the Adam optimizer
optimizer = torch.optim.Adam(clip_model.ecg_encoder.parameters())

In [40]:
num_params = sum(p.numel() for p in clip_model.parameters())
print("Number of parameters: ", num_params)

Number of parameters:  34223077


In [41]:
# from tqdm.notebook import tqdm

# num_epochs = 3  # Define the number of epochs

# # Start the training loop
# for epoch in range(num_epochs):
#     # Set the model to training mode
#     clip_model.train()

#     # Initialize the total loss for this epoch
#     total_loss = 0.0

#     # Create a progress bar
#     progress_bar = tqdm(range(len(sample_train_set_250)), desc="Training")

#     # Iterate over the training data
#     for i in progress_bar:
#         # Get the ECG signal and diagnosis
#         signal = torch.from_numpy(sample_train_set_250[i][1]['val']).float().to(device)
#         print("Input size to batch normalization layer: ", signal.size())
#         diagnosis = sample_processed_train_df_250['dx_modality'][i]

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         loss = clip_model(signal, diagnosis)

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()

#         # Update the total loss
#         total_loss += loss.item()

#         # Update the progress bar
#         progress_bar.set_postfix({'epoch': epoch+1, 'loss': total_loss/(i+1)}, refresh=True)

#     # Compute the average loss for this epoch
#     avg_loss = total_loss / len(sample_train_set_250)

#     # Update the description of the progress bar
#     progress_bar.set_description(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

#     # Print the results for this epoch
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Training:   0%|          | 0/250 [00:00<?, ?it/s]

Input size to batch normalization layer:  torch.Size([3, 5000])


RuntimeError: ignored

In [47]:
# Move the model to the GPU
clip_model = clip_model.to(device)

num_epochs = 3

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    clip_model.train()

    # Loop over each batch in the training set
    for i in range(len(sample_train_set_250)):
        # Get the ECGs and diagnoses
        ecgs = torch.tensor(sample_train_set_250[i][1]['val']).float().to(device)

        # Reshape the ECGs
        ecgs = ecgs.view(1, 3, -1)

        # Get diagnoses as a list of strings
        diagnoses_strings = sample_processed_train_df_250['dx_modality'][i]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        loss = clip_model(ecgs, diagnoses_strings)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Print the loss for this batch
        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(sample_train_set_250)}, Loss: {loss.item()}")

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

RuntimeError: ignored